
## How Can We Discover the Optimal Crossing Strategy?

This is where algorithmic thinking comes in. When we are not given the optimal steps, we must explore possible **states** and transitions between them.

### Combinatorial State Space

- Each **state** consists of who is on the left side, right side, and the flashlight's location.
- The **initial state** has all four people on the left with the flashlight.
- A **transition** is a valid move: either one or two people cross or return, depending on where the flashlight is.
- Each transition adds time (equal to the slowest person in the crossing group).
- The **goal** is to get everyone to the right side with the flashlight.

### Solving with Breadth-First Search (BFS)

We use BFS to:
1. Explore all valid sequences of moves.
2. Track time spent and avoid re-visiting already seen configurations.
3. Return the path that gets everyone across with the **minimum total time**.

This is a classic example of **combinatorial search** in discrete optimization.


In [2]:
from collections import deque

# Crossing times
times = {'A': 1, 'B': 2, 'C': 5, 'D': 10}

# Initial state: (left_side, right_side, flashlight_side, time_spent, path)
initial_state = (frozenset(['A', 'B', 'C', 'D']), frozenset(), 'left', 0, [])

# BFS setup
queue = deque([initial_state])
visited = dict()  # track best known cost to a state

min_time = float('inf')
best_path = []

while queue:
    left, right, light, time_spent, path = queue.popleft()

    # Use sorted tuple to normalize state regardless of ordering
    state_id = (frozenset(left), frozenset(right), light)
    if state_id in visited and visited[state_id] <= time_spent:
        continue
    visited[state_id] = time_spent

    # Goal check
    if len(left) == 0 and light == 'right':
        if time_spent < min_time:
            min_time = time_spent
            best_path = path
        continue

    if light == 'left':
        # Choose all 2-person combinations from left
        people = list(left)
        for i in range(len(people)):
            for j in range(i + 1, len(people)):
                a, b = people[i], people[j]
                cost = max(times[a], times[b])
                new_left = left - {a, b}
                new_right = right | {a, b}
                new_path = path + [(a, b)]
                queue.append((new_left, new_right, 'right', time_spent + cost, new_path))
    else:
        # Only allow 1 person to return
        for p in right:
            cost = times[p]
            new_left = left | {p}
            new_right = right - {p}
            new_path = path + [(p,)]
            queue.append((new_left, new_right, 'left', time_spent + cost, new_path))

# Final output
print("Optimal total time:", min_time, "minutes")
print("Optimal steps:")
for step in best_path:
    print(" ", step)


Optimal total time: 17 minutes
Optimal steps:
  ('A', 'B')
  ('A',)
  ('D', 'C')
  ('B',)
  ('A', 'B')
